# PAC

PAC analysis for sensorimotor and frontal areas

All epochs are analysed

In [1]:
import mne
import os
from utils import check_paths
import numpy as np
import pandas as pd
import pickle
from scipy.io import loadmat

from tensorpac import Pac, EventRelatedPac, PreferredPhase
from tensorpac.utils import PeakLockedTF, PSD, ITC, BinAmplitude



import matplotlib.pyplot as plt
%matplotlib qt

In [ ]:
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
group_save_path = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\Y group'
sub_name = 'ALL_subs'
task = '_MAIN' # ['_BL', '_MAIN']
block_name = '_adaptation' # ['_baseline', '_adaptation']
task_stage = '_go' # '_plan' or '_go'
epochs_all_subs = mne.read_epochs(os.path.join(group_save_path, f"{sub_name}{task}_epochs{task_stage}{block_name}_ALL-epo.fif"), preload=True)
epochs_all_subs

In [37]:
pac_figs_dir = os.path.join(eeg_data_dir, 'figures', 'PAC', task, block_name, 'NEW')
check_paths(pac_figs_dir)
pac_dir = os.path.join('D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\Y group', 'PAC', task, task_stage, block_name)
check_paths(pac_dir)

In [38]:
# Extract only EEG channel names from the Epochs object
eeg_channel_names = epochs_all_subs.copy().pick("eeg").ch_names
# print(eeg_channel_names)

# Select sensorimotor channels
choi = ['Fp1', 'Fp2', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'AF3', 'AF4', 'AF7', 'AF8',
        'FC1', 'FC2', 'FC3', 'FC4', 'FC5', 'FC6', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6']

epochs_all_subs.pick(choi)

<EpochsFIF | 2835 events (all good), -0.5 – 0.7 s (baseline -0.5 – 0.7 s), ~390.0 MB, data loaded, with metadata,
 'go_on': 2835>

In [39]:
sf = epochs_all_subs.info['sfreq']  # Sampling frequency
times = epochs_all_subs.times

In [40]:
# set freqs for PAC analysis
width, step = 3, .2
theta = [3, 9]
alpha = [6, 14]
lower_gamma = (30, 50, 2, 0.5)
higher_gamma = (50, 80, 2, 0.5)

In [ ]:
pac_method = 1 # 1 - MVL, 2 - MI, 3 - HR, 4 - ndPAC, 5 - PLV, 6 - GCPAC
surr_method = 2 # 0 - no surr, 1 - swap phase, 2 - swap amp, 3 - time lag
norm_method = 4 # 0 - no norm, 1 - subtract, 2 - divide, 3 - subtr+divide, 4 - zscore

for chan in range(len(choi)): # range(len(choi))
    # Load the data from an MNE Epochs object (e.g., shape: epochs x channels x time)
    data = epochs_all_subs.get_data()[:,chan,:]  # Shape: (n_epochs, n_channels, n_times)

    print(f"DATA: (n_trials, n_times)={data.shape}; SAMPLING FREQUENCY={sf}Hz; "
        f"TIME VECTOR: n_times={len(times)}")
    

    for f_pha in [theta, alpha]: # [theta, alpha]
        print(f_pha)

        if f_pha == theta:
            phase_name = 'theta'
            amp_name = 'lower gamma'
            f_amp = lower_gamma
        else:
            phase_name = 'alpha'
            amp_name = 'higher gamma'
            f_amp = higher_gamma

        # Event-related PAC - plot
        rp_obj = EventRelatedPac(f_pha=f_pha, f_amp=f_amp)
        erpac = rp_obj.filterfit(sf, data, method='gc', smooth=100)

        plt.figure(figsize=(8, 6))
        ax = rp_obj.pacplot(erpac.squeeze(), times, rp_obj.yvec, xlabel='Time',
                    ylabel='Amplitude frequency (Hz)',
                    title=f'{choi[chan]}: Event-Related PAC {phase_name}-{amp_name}',
                    fz_labels=15, fz_title=18)
        fig = ax.get_figure()
        fig.savefig(os.path.join(pac_figs_dir, f"{sub_name}_ERPac{task_stage}_{phase_name}{choi[chan]}.png"), dpi=300, bbox_inches='tight')
        plt.close(fig)

        # # PAC for planning
        phase = tuple((*f_pha, width, step))
        print(phase)

        p_obj = Pac(idpac=(pac_method, surr_method, norm_method), f_pha=phase, f_amp=f_amp) # 2 - MODULATION INDEX (MI)
        # extract all of the phases and amplitudes
        pha_p = p_obj.filter(sf, data, ftype='phase')
        amp_p = p_obj.filter(sf, data, ftype='amplitude')

        # define time indices of planning
        time_all = slice(250, 500) # 0s-0.5s

        # define phase / amplitude during planning
        pha_prep, amp_prep = pha_p[..., time_all], amp_p[..., time_all]

        # compute PAC inside planning
        pac_prep = p_obj.fit(pha_prep, amp_prep).mean(-1)

        vmax = np.max(pac_prep.max())
        kw = dict(vmax=vmax, vmin=.04, cmap='viridis')
        plt.figure(figsize=(8, 6))
        ax = p_obj.comodulogram(pac_prep, title=f"PAC {task_stage}", **kw)
        fig = ax.get_figure()
        plt.ylabel('')
        plt.tight_layout()
        fig.savefig(os.path.join(pac_figs_dir, f"{sub_name}_PAC{task_stage}_{phase_name}_{choi[chan]}.png"), dpi=300, bbox_inches='tight')
        # ValueError: Format 'fig' is not supported (supported formats: eps, jpeg, jpg, pdf, pgf, png, ps, raw, rgba, svg, svgz, tif, tiff, webp)
        plt.close(fig)


        # # STATISTICS
        # computing the permutations
        p_obj.idpac = (pac_method, surr_method, norm_method) 
        # compute pac and 200 surrogates
        pac_prep = p_obj.fit(pha_p[..., time_all], amp_p[..., time_all], n_perm=200,
                             random_state=0)
        # get the p-values
        mcp = 'maxstat'
        pvalues = p_obj.infer_pvalues(p=0.05, mcp=mcp)

        # sphinx_gallery_thumbnail_number = 7
        plt.figure(figsize=(8, 6))
        title = (f'''Significant {phase_name}-{amp_name} coupling\n
                 occurring during the motor planning phase\n
                 (p<0.05, {mcp}-corrected for multiple comparisons''')

        # plot the non-significant pac in gray
        pac_prep_ns = pac_prep.mean(-1).copy()
        pac_prep_ns[pvalues < .05] = np.nan
        p_obj.comodulogram(pac_prep_ns, cmap='gray', vmin=np.nanmin(pac_prep_ns),
                           vmax=np.nanmax(pac_prep_ns), colorbar=False)
        
        # plot the significant pac in color
        pac_prep_s = pac_prep.mean(-1).copy()
        pac_prep_s[pvalues >= .05] = np.nan
        ax = p_obj.comodulogram(pac_prep_s, cmap='Spectral_r', vmin=np.nanmin(pac_prep_s),
                           vmax=np.nanmax(pac_prep_s), title=title)
        plt.gca().invert_yaxis()
        fig = ax.get_figure()
        fig.savefig(os.path.join(pac_figs_dir, f"{sub_name}_PAC_STATS{task_stage}_{phase_name}_{choi[chan]}.png"), dpi=300, bbox_inches='tight')
        plt.close(fig)

        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        # np.save(os.path.join(pac_dir, f"{sub_name}_ERPac_{phase_name}_{choi[chan]}.npy"), rp_obj)
        # np.save(os.path.join(pac_dir, f"{sub_name}_PAC{task_stage}_{phase_name}_{choi[chan]}.npy"), p_obj)
        # np.save(os.path.join(pac_dir, f"{sub_name}_PAC_STATS{task_stage}_{phase_name}_{choi[chan]}_NON-SIG.npy"), pac_prep_ns)
        # np.save(os.path.join(pac_dir, f"{sub_name}_PAC_STATS{task_stage}_{phase_name}_{choi[chan]}_SIG.npy"), pac_prep_s)

        # Save the object using pickle
        # with open(os.path.join(pac_dir, f"{sub_name}_ERPac{task_stage}_{phase_name}_{choi[chan]}.pkl"), 'wb') as file:
        #     pickle.dump(rp_obj, file)
        with open(os.path.join(pac_dir, f"{sub_name}_PAC{task_stage}_{phase_name}_{choi[chan]}.pkl"), 'wb') as file:
            pickle.dump(p_obj, file)
        # with open(os.path.join(pac_dir, f"{sub_name}_PAC_STATS{task_stage}_{phase_name}_{choi[chan]}_NON-SIG.pkl"), 'wb') as file:
        #     pickle.dump(pac_prep_ns, file)
        with open(os.path.join(pac_dir, f"{sub_name}_PAC_STATS{task_stage}_{phase_name}_{choi[chan]}_SIG.pkl"), 'wb') as file:
            pickle.dump(pac_prep_s, file)


Notes on PAC methods

1 - WARNING | MVL is amplitude dependent which means that if the amplitude increases, MVL also increases. You should select a normalization method for correcting this limitation (e.g idpac=(1, 2, 4))
WARNING | If you want to normalize the estimated PAC, you should select a surrogate method (second digit of `idpac`). Normalization ignored.

400 -  77 pvalues = p_obj.infer_pvalues(p=0.05, mcp=mcp) -> AttributeError: 'NoneType' object has no attribute 'mean'

______

Reading files

In [ ]:
rp_obj1 = np.load(os.path.join(pac_dir, f"{sub_name}_ERPac_{phase_name}_{eeg_channel_names[chan]}.npy"), allow_pickle=True)
rp_obj1

array(<tensorpac.pac.EventRelatedPac object at 0x0000017A65F9D100>,
      dtype=object)

In [ ]:
# Load the saved object
with open(os.path.join(pac_dir, f"{sub_name}_ERPac_{phase_name}_{eeg_channel_names[chan]}.pkl"), 'rb') as file:
    rp_obj1 = pickle.load(file)

# The object is now restored and ready to use
print(rp_obj1)

Gaussian-Copula ERPAC
